### Filter VCF files 

1. Get rid of labels on the top of the file (identified by ## at the start of each line) 
2. Change file to dataframe (put all files in one big dataframe)
3. Filter by Quality 
4. Filter by Chromosome 
5. Filter by Duplicates 
6. Write to Excel File 

In [2]:
import pandas as pd 
import numpy as np 
import csv
import os 
from collections import Counter
import chardet 
os.chdir("/Users/jadetakakuwa/Desktop/SungLab/currentData/Tumefaciens/agroVCF")

In [3]:
#Function to create a dataframe from a vcf file 
def vcfDataFrame(path_to_vcf):
    #Read in file 
    with open (path_to_vcf, 'r') as vcf_file: 
        read_vcf = csv.reader(vcf_file, delimiter='\t')
        for line in read_vcf: 
                
            # If line is part of the header, skip 
            if line[0].startswith("##"):
                pass
                
            #Discard last column name, which is the unique sample name 
            #Create empty dataframe with column names to match the VCF except for the last column 
            elif line[0].startswith("#CHROM"):
                column_names = line[0:len(line)-1]
                vcf_dF = pd.DataFrame(columns= column_names)

            else:
                #Add information in list to the dataframe 
                vcf_dF.loc[len(vcf_dF)] = line[0:len(line)-1]
        
        #Create a Sample_name column 
        #Fill in Sample_name for sample name using the original VCF file name 
        path_list = path_to_vcf.split("/")
        new_column_value = path_list[len(path_list)-1][0:len(path_list)-19]
        vcf_dF["Sample_name"] = new_column_value
        
        
        return vcf_dF


In [4]:
#Implementing vcfDataFrame function for a directory of VCF files 
def create_vcf_dataFrame(path_to_files, path_to_new_excel):
    
    #Create an empty dataframe with the necessary columns to match new VCF dataframe 
    final_dF = pd.DataFrame(columns=['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT',
       'Sample_name'])
    
    #Write a new excel file with empty dataframe
    #This will be used further downstream 
    #The path to the excel file should include the new file name 
    final_dF.to_excel(path_to_new_excel)
    
    #Loop over all VCF files and use vcfDataFrame to convert to dataframes 
    #Add each new dataframe to the empty dataframe to have a dataframe including all sample info 
    for vcf in os.listdir(path_to_files):
        path_to_vcf = str(path_to_files) + "/" + str(vcf)
        new_dF = vcfDataFrame(path_to_vcf)
        final_dF = pd.concat([new_dF, final_dF])
        
    return final_dF




In [5]:
#Filter for quality and drop duplicates 
#Discard position if quality is less than 60 
#Discard position if it is duplicated in 5 or more samples
#Put chromosome of interest as a parameter and discard all other chromosome information 
def filter_vcf_dataFrame(vcf_dataFrame, chromosome_info): 

    #Quality filter 
    vcf_dataFrame["QUAL"] = pd.to_numeric(vcf_dataFrame["QUAL"], downcast="float")
    vcf_dataFrame = vcf_dataFrame[(vcf_dataFrame["QUAL"] > 60.0)]

    #Removing unnecessary chromosomes - dependent on species 
    vcf_dataFrame = vcf_dataFrame[(vcf_dataFrame["#CHROM"] == chromosome_info)]

    #Filter for duplicates 
    #If a position is shared by 5 or more samples, discard in all samples
    
    #Make a dataframe with only unique values 
    vcf_dataFrame_nodup = vcf_dataFrame.drop_duplicates(subset=["POS"], keep = False)
    
    #Create empty list to store duplicates 
    list_of_dup = []
    
    #Iterate through dataframe, and add duplicated postions to the duplicate list 
    for index, row in vcf_dataFrame.iterrows():
        if row["POS"] not in vcf_dataFrame_nodup["POS"]: 
            list_of_dup.append(row["POS"])

    #Count the number of duplicates per position in the duplicate list  
    count_dup = Counter(list_of_dup)

    #For every position and count, if the count is greater than 5, drop the position from the dataframe
    for key, value in count_dup.items():
        if value >= 5: 
            vcf_dataFrame = vcf_dataFrame.loc[vcf_dataFrame["POS"] != key]
        else: 
            pass
        
    return vcf_dataFrame



In [17]:
#Write the new filtered dataframe to the empty excel sheet created above 
def createExcel(filtered_vcf, path_to_new_excel):
    
    #Create excel file with samples for each sheet 
    samples = filtered_vcf.Sample_name.unique()
    
    #Create a new sheet containing information from each sample 
    for sample_name in samples:
        sample = filtered_vcf.loc[filtered_vcf["Sample_name"] == sample_name]
        with pd.ExcelWriter(path_to_new_excel, engine="openpyxl", mode='a') as writer:  
            sample.to_excel(writer, sheet_name= sample_name )
            
createExcel(tumFiltered, '/Users/jadetakakuwa/Desktop/SungLab/currentData/Tumefaciens/tum_new_vcf.xlsx')